<a href="https://colab.research.google.com/github/Ishan0704/Suspicious-Transaction-Detection/blob/main/RNN_Bitcoin_Heist_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/BitcoinHeistData.csv')
df

,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,1.000500e+08,princetonCerber
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,1.000000e+08,princetonLocky
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,2.000000e+08,princetonCerber
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,7.120000e+07,princetonCerber
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,2.000000e+08,princetonLocky
...,...,...,...,...,...,...,...,...,...,...
2916692,12D3trgho1vJ4mGtWBRPyHdMJK96TRYSry,2018,330,0,0.111111,1,0,1,1.255809e+09,white
2916693,1P7PputTcVkhXBmXBvSD9MJ3UYPsiou1u2,2018,330,0,1.000000,1,0,1,4.409699e+07,white
2916694,1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw,2018,330,2,12.000000,6,6,35,2.398267e+09,white
2916695,15iPUJsRNZQZHmZZVwmQ63srsmughCXV4a,2018,330,0,0.500000,1,0,1,1.780427e+08,white


***STEP 1 - Pre-process the data***

Scale the features for better model training.




In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])


In [ ]:
df

,address,year,day,length,weight,count,looped,neighbors,income,label,label_encoded
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,1.000500e+08,princetonCerber,26
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,1.000000e+08,princetonLocky,27
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,2.000000e+08,princetonCerber,26
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,7.120000e+07,princetonCerber,26
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,2.000000e+08,princetonLocky,27
...,...,...,...,...,...,...,...,...,...,...,...
2916692,12D3trgho1vJ4mGtWBRPyHdMJK96TRYSry,2018,330,0,0.111111,1,0,1,1.255809e+09,white,28
2916693,1P7PputTcVkhXBmXBvSD9MJ3UYPsiou1u2,2018,330,0,1.000000,1,0,1,4.409699e+07,white,28
2916694,1KYiKJEfdJtap9QX2v9BXJMpz2SfU4pgZw,2018,330,2,12.000000,6,6,35,2.398267e+09,white,28
2916695,15iPUJsRNZQZHmZZVwmQ63srsmughCXV4a,2018,330,0,0.500000,1,0,1,1.780427e+08,white,28


In [ ]:
# Select features and target
features = ['year', 'day', 'length', 'weight', 'count', 'looped', 'neighbors', 'income']
X = df[features].values
y = df['label_encoded'].values


***Normalize the features***

Scale the features for better model training.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


***Reshape the data for RNN***

Reshape the data into 3D arrays required by RNNs.

In [ ]:
# Reshape data for RNN
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


***STEP 2 - Define RNN Model***

Create the RNN model using TensorFlow/Keras.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# Define the model
model = Sequential()

# Add layers
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(set(y)), activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


***STEP 3 - Train the model***

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/20
58334/58334 ━━━━━━━━━━━━━━━━━━━━ 301s 5ms/step - accuracy: 0.9855 - loss: 0.1162 - val_accuracy: 0.9859 - val_loss: 0.0695
Epoch 2/20
58334/58334 ━━━━━━━━━━━━━━━━━━━━ 311s 5ms/step - accuracy: 0.9857 - loss: 0.0719 - val_accuracy: 0.9859 - val_loss: 0.0697
Epoch 3/20
58334/58334 ━━━━━━━━━━━━━━━━━━━━ 268s 5ms/step - accuracy: 0.9857 - loss: 0.0710 - val_accuracy: 0.9859 - val_loss: 0.0682
Epoch 4/20
58334/58334 ━━━━━━━━━━━━━━━━━━━━ 268s 5ms/step - accuracy: 0.9858 - loss: 0.0700 - val_accuracy: 0.9859 - val_loss: 0.0689
Epoch 5/20
58334/58334 ━━━━━━━━━━━━━━━━━━━━ 291s 5ms/step - accuracy: 0.9858 - loss: 0.0702 - val_accuracy: 0.9859 - val_loss: 0.0680
Epoch 6/20
58334/58334 ━━━━━━━━━━━━━━━━━━━━ 273s 5ms/step - accuracy: 0.9856 - loss: 0.0706 - val_accuracy: 0.9859 - val_loss: 0.0681
Epoch 7/20
58334/58334 ━━━━━━━━━━━━━━━━━━━━ 330s 5ms/step - accuracy: 0.9857 - loss: 0.0702 - val_accuracy: 0.9859 - val_loss: 0.0677
Epoch 8/20
58334/58334 ━━━━━━━━━━━━━━━━━━━━ 340s 5ms/step - ac

***STEP 4 - Evaluate the Model***

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")


Test Accuracy: 0.99


***STEP 5 - Make Prediciton***

In [ ]:
import numpy as np

# Predict labels
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Decode the predicted labels
decoded_labels = label_encoder.inverse_transform(predicted_labels)
print(decoded_labels[:10])  # Print first 10 predictions


18230/18230 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step
['white' 'white' 'white' 'white' 'white' 'white' 'white' 'white' 'white'
 'white']


In [ ]:
# Save the model
model.save('rnn_bitcoin_heist_model.h5')


In [ ]:
model.save('my_model.keras')

***Example***

In [ ]:
import numpy as np

# New data sample (replace 'princetonCerber' with its features only)
new_sample = np.array([[2017, 11, 18, 0.008333333, 1, 0, 2, 100050000]])

# Normalize the new sample using the same scaler used during training
new_sample = scaler.transform(new_sample)

# Reshape the data for RNN (samples, timesteps, features)
new_sample = new_sample.reshape((new_sample.shape[0], 1, new_sample.shape[1]))


In [ ]:
# Predict the label
predicted_label_encoded = np.argmax(model.predict(new_sample), axis=1)

# Decode the predicted label
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)
print(f"Predicted Label: {predicted_label[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted Label: white


***2ND Example***

In [ ]:
# Select feature columns and target
features = ['year', 'day', 'length', 'weight', 'count', 'looped', 'neighbors', 'income']
X = df[features].values  # Feature matrix
y = df['label_encoded'].values  # Target labels (encoded earlier)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the features
X = scaler.fit_transform(X)


In [ ]:
import joblib

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [ ]:
# Load the scaler
scaler = joblib.load('scaler.pkl')


In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Reshape for RNN
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [ ]:
# New sample data
new_sample = np.array([[2017, 11, 18, 0.008333333, 1, 0, 2, 100050000]])

# Normalize using the previously saved scaler
new_sample = scaler.transform(new_sample)

# Reshape for RNN
new_sample = new_sample.reshape((new_sample.shape[0], 1, new_sample.shape[1]))


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('my_model.keras')


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Predict the label
predicted_label_encoded = np.argmax(model.predict(new_sample), axis=1)

# Decode the predicted label back to its original form
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)

# Display the result
print(f"Predicted Label: {predicted_label[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step
Predicted Label: white


***Evaluation Parameters***

In [31]:
# Predict probabilities for the test set
y_pred_probs = model.predict(X_test)

# Convert probabilities to predicted class labels
y_pred = np.argmax(y_pred_probs, axis=1)


18230/18230 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step


In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Accuracy: 0.99
Precision: 0.97
Recall: 0.99
F1 Score: 0.98


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
